# Environment

In [1]:
import higlass
from higlass.client import View, Track, CombinedTrack
from cooler import Cooler
from higlass.tilesets import cooler, beddb, chromsizes, bigwig, Tileset
import clodius
import os
import os.path as path
import pandas as pd
import numpy as np
import itertools
import negspy.coordinates as nc
import matplotlib.pyplot as plt
import scipy.stats as stats
from tqdm import tqdm

def bed2ddb(filepath, uuid=None, **kwargs):
    from clodius.tiles.utils import tiles_wrapper_2d
    from clodius.tiles.bed2ddb import get_2d_tileset_info, get_2D_tiles
    return Tileset(
        uuid=uuid,
        tileset_info=lambda: get_2d_tileset_info(filepath),
        tiles=lambda tids: tiles_wrapper_2d(
            tids,
            lambda z,x,y: get_2D_tiles(filepath, z, x, y)[(x, y)]
        ),
        **kwargs
    )


# Load annotations

In [2]:
genes = beddb("../2019-10-24_higlass/Data/hg38/gene-annotations-hg38.beddb")
chrom_sizes = chromsizes("../2019-10-24_higlass/hg38.chrom.sizes")
label_font_size = 18
chr_label_size = 30
annots_size = 150

chrom_labels = {
    p: Track(
        track_type=l + "-chromosome-labels",
        tileset=chrom_sizes,
        position=p,
        height=chr_label_size,
        width=chr_label_size,
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    ) for p, l in zip(["top", "bottom", "left", "right"], ["horizontal", "horizontal", "vertical", "vertical"])
}

gene_annots = {
    p: Track(
        track_type=l + "-gene-annotations",
        tileset=genes,
        position=p,
        height=annots_size,
        width=annots_size,
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    ) for p, l in zip(["top", "bottom", "left", "right"], ["horizontal", "horizontal", "vertical", "vertical"])
}

# genome coordinates
hg38 = nc.get_chrominfo("hg38")

# Data

## Contact matrices

In [3]:
tumour_metadata = pd.read_csv(path.join("..", "..", "Data", "External", "LowC_Samples_Data_Available.tsv"), sep="\t", header=0)
tumour_metadata = tumour_metadata.loc[tumour_metadata.Include == "Yes", :]
tumour_metadata["SampleID"] = ["PCa" + str(i) for i in tumour_metadata["Sample ID"]]

tumour_samples = tumour_metadata["SampleID"].tolist()
t2e_samples = tumour_metadata.loc[tumour_metadata["T2E Status"] == "Yes", "SampleID"].tolist()
nont2e_samples = tumour_metadata.loc[tumour_metadata["T2E Status"] == "No", "SampleID"].tolist()

benign_metadata = pd.read_csv(path.join("..", "..", "Data", "Raw", "191220_A00827_0104_AHMW25DMXX_HiC", "config.tsv"), sep="\t", header=0)
benign_metadata = benign_metadata.loc[benign_metadata.Include == "Yes", :]
benign_samples = benign_metadata["Sample"].tolist()

cell_line_metadata = pd.read_csv(path.join("..", "..", "Data", "External", "Rhie_2019", "config.tsv"), sep="\t", header=0)
cell_line_samples = cell_line_metadata["Run_Accession"].tolist()

all_samples = tumour_samples + benign_samples + cell_line_samples
metadata = pd.read_csv(path.join("..", "2020-01-15_TAD-aggregation", "config.tsv"), sep="\t", index_col=False, header=0)
cooler_files = (
    [path.join("..", "..", "Data", "Processed", "2019-06-18_PCa-LowC-sequencing", "Contacts", s + ".mcool") for s in tumour_samples + benign_samples]
    + [path.join("..", "..", "Data", "External", "Rhie_2019", "Contacts", s + ".mcool") for s in cell_line_samples]
)
lowc_tilesets = {s: cooler(f) for s, f in zip(all_samples, cooler_files)}

resolutions = [
    1000, 2000, 3000, 4000, 5000,
    10000, 20000, 30000, 40000, 50000,
    100000, 200000, 300000, 400000, 500000,
    1000000, 2000000, 3000000, 4000000, 5000000
][::-1]

min_resolution = 40000
heatmap_size = 250
colour_range = [
    "rgba(65, 105, 225, 1.0)",
    "rgba(255, 250, 250, 1.0)",
    # "rgba(240, 128, 128, 1.0)",
    "rgba(255, 25, 25, 1.0)"
]
lowc_heatmaps = {
    s: {
        p: Track(
            track_type=l + "heatmap",
            position=p,
            tileset=lowc_tilesets[s],
            filetype="cooler",
            height=heatmap_size,
            options={
                "maxZoom": str(resolutions.index(min_resolution)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
                "name": metadata.loc[metadata["SampleID"] == s, "Label"].values[0],
                "colorRange": colour_range,
            },
        ) for p, l in zip(
            ["top", "bottom", "left", "right", "center"],
            ["horizontal-", "horizontal-", "vertical-", "vertical-", ""]
        )
    } for s in all_samples
}

## Loops

In [4]:
TRACK_DIR = path.join("Tracks")

loop_tilesets_2D = bed2ddb(path.join(TRACK_DIR, "tumour-specific-loops.bed2ddb"), name="Tumour-Specific Loops")
loop_tracks_2D = {
   "top": Track(
        track_type="horizontal-2d-rectangle-domains",
        position="top",
        tileset=loop_tilesets_2D,
        filetype="bed2ddb",
        options={
            "showMousePosition": True,
            "name": loop_tilesets_2D.tileset_info()["name"],
        },
    ),
    "bottom": Track(
        track_type="horizontal-2d-rectangle-domains",
        position="bottom",
        tileset=loop_tilesets_2D,
        filetype="bed2ddb",
        options={
            "showMousePosition": True,
            "name": loop_tilesets_2D.tileset_info()["name"],
        },
    ),
    "centre": Track(
        track_type="2d-rectangle-domains",
        position="center",
        tileset=loop_tilesets_2D,
        filetype="bed2ddb",
        width=500,
        height=500,
        options={
            "showMousePosition": True,
            "name": loop_tilesets_2D.tileset_info()["name"],
            "flipDiagonal": "copy",
        }
    ),
}

# Figures

In [32]:
test_view = View(
    tracks=[
        gene_annots["top"],
        chrom_labels["top"],
        #loop_tracks_2D["top"],
        #loop_tracks_2D["top"],
        CombinedTrack([lowc_heatmaps["PCa13266"]["top"], loop_tracks_2D["centre"],]),
        #lowc_heatmaps["Benign-Prostate-1595983"]["top"],
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chrX", 68e6 - 3e6, hg38),
        nc.chr_pos_to_genome_pos("chrX", 68e6 + 3e6, hg38),
    ]
)

d, s, v = higlass.display(
    views=[test_view],
    # value_scale_syncs=[
    #     [(test_view, lowc_heatmaps[s]["top"]) for s in ["PCa13266", "SRR7702334", "Benign-Prostate-1595983", "SRR8446383"]]
    # ],
)
#d
loop_tilesets_2D

# Aggregate Peak Analysis

In [8]:
LOOP_DIR = "Loops"

# load calculations
agg_loop_matrices = {
    (loop_type, sample_type): np.load(
        path.join(LOOP_DIR, "{loop_type}-loops.{sample_type}-samples.agg.npz".format(loop_type=loop_type, sample_type=sample_type))
    )
    for loop_type in ["tumour", "benign", "shared"]
    for sample_type in ["tumour", "benign"]
}

# make plots for each comparison
for (loop_type, sample_type) in agg_loop_matrices.keys():
    # plot differences between mutant and nonmutant/benign samples
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(111)
    ax.set_title("{} loops".format(sample_type))
    plt.imshow(agg_loop_matrices[(loop_type, sample_type)], vmin=0.01, vmax=0.07, cmap="Reds")
    ax.set_aspect("equal")
    fig.savefig(
        "Plots/{loop_type}-loops.{sample_type}-apa.pdf".format(
            loop_type=t, sample_type=sample_type
        )
    )